# Advent of Code 2023

## Day 03

https://adventofcode.com/2023/day/3

In [168]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2023, day=3)

In [99]:
example_schematic = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

class WrongEngine:
    schematic: list[list]
    
    def _find_neighbor(self, i: int, j: int) -> list[(int, int, int)]:
        print(f'Looking around {self.schematic[i][j]} ({i}, {j})')
        
        neighbors: list((int, int)) = []
        
        STEP = 1

        UPPER_BOUND = i - STEP
        LOWER_BOUND = i + STEP
        LEFT_BOUND = j - STEP
        RIGHT_BOUND = j + STEP

        row_indices = range(UPPER_BOUND, LOWER_BOUND + 1)
        col_indices = range(LEFT_BOUND, RIGHT_BOUND + 1)

        for row_indx in row_indices:
            row_in_schematic = row_indx >= 0 and row_indx < len(self.schematic)
            if not row_in_schematic:
                continue

            cols = []
            # First and last row include all columns
            if row_indx == UPPER_BOUND or row_indx == LOWER_BOUND:
                cols = col_indices
            else:
                cols = [LEFT_BOUND, RIGHT_BOUND]

            for col_indx in cols:
                col_in_schematic = col_indx >= 0 and col_indx < len(self.schematic[row_indx])
                if not col_in_schematic:
                    continue
                value = self.schematic[row_indx][col_indx] 
                print(f'found {value if value else '.'} ({row_indx}, {col_indx})')

        return neighbors

    def __init__(self, schematic_raw: str) -> None:
        self.schematic = [list(i) for i in schematic_raw.split('\n')]

    def __str__(self) -> str:
        return '\n'.join(''.join(i if i else '.' for i in row) for row in self.schematic)
    
print(Engine(example_schematic))

467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..


In [100]:
e = Engine(example_schematic)
e._find_neighbor(0,0)

Looking around 4 (0, 0)
row is out of range
col is out of range
found 6 (0, 1)
col is out of range
found . (1, 0)
found . (1, 1)


OOf!! Just re-read the instructions now that I'm ready to solve. It's not looking for the adjacent numbers as a single digit, it considers contigious numbers on a line to be the number! Back to the drawing board :(

In [172]:
import re

example_schematic = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

class Engine:
    schematic: list[list]

    def model_numbers(self):
        model_numbers = []
        for i, row in enumerate(self.schematic):
            # I think I could do this with a list comprehension filter, but it would be more difficult to read IMHO
            for m in re.finditer(r'\d+', ''.join(row)):
                match_added = False
                for j in range(int(m.span(0)[0]), int(m.span(0)[1])):
                    if self._has_neighbor_symbol(i, j) and not match_added:
                        model_numbers.append(int(m.group()))
                        match_added = True
        return model_numbers
    
    def _has_neighbor_symbol(self, i: int, j: int) -> list[(int, int, int)]:
        neighbors: list((int, int)) = []
        
        STEP = 1

        UPPER_BOUND = i - STEP
        LOWER_BOUND = i + STEP
        LEFT_BOUND = j - STEP
        RIGHT_BOUND = j + STEP

        row_indices = range(UPPER_BOUND, LOWER_BOUND + 1)
        col_indices = range(LEFT_BOUND, RIGHT_BOUND + 1)

        for row_indx in row_indices:
            row_in_schematic = row_indx >= 0 and row_indx < len(self.schematic)
            if not row_in_schematic:
                continue

            cols = []

            # First and last row include all columns
            if row_indx == UPPER_BOUND or row_indx == LOWER_BOUND:
                cols = col_indices
            else:
                cols = [LEFT_BOUND, RIGHT_BOUND]

            for col_indx in cols:
                col_in_schematic = col_indx >= 0 and col_indx < len(self.schematic[row_indx])
                if not col_in_schematic:
                    continue
                value = self.schematic[row_indx][col_indx] 
                if re.match(r'[^\d\.]', value):
                    # print(f'"{value}" is not a digit!')
                    # print(f'"{self.schematic[i][j]}" @ ({i}, {j}) has a special symbol "{value}" @ ({row_indx}, {col_indx})')
                    return True

                # print(f'found {value if value else '.'} ({row_indx}, {col_indx})')
        # print(f'"{self.schematic[i][j]}" @ ({i}, {j}) didn\'t have a neighbor with a special symbol')
        return False

    def __init__(self, schematic_raw: str) -> None:
        self.schematic = [list(i) for i in schematic_raw.split('\n')]

    def __str__(self) -> str:

        return '\n'.join(''.join(row) for row in self.schematic)
    
e = Engine(example_schematic)
# print(e)
# e._has_neighbor_symbol(0, 5)
print(sum(e.model_numbers()))

4361


In [170]:
e_a = Engine(puzzle.input_data)
print(f'Solution A: {sum(e_a.model_numbers())}')

Solution A: 498559


In [188]:
import re
import math
from collections import defaultdict

example_schematic = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

class Engine:
    schematic: list[list]

    def model_numbers(self):
        return [num for num, _ in self._model_numbers()]

    def _model_numbers(self):
        model_numbers = []
        for i, row in enumerate(self.schematic):
            # I think I could do this with a list comprehension filter, but it would be more difficult to read IMHO
            for m in re.finditer(r'\d+', ''.join(row)):
                match_added = False
                for j in range(int(m.span(0)[0]), int(m.span(0)[1])):
                    neighbor = self._has_neighbor_symbol(i, j)
                    if neighbor and not match_added:
                        model_numbers.append((int(m.group()), neighbor))
                        match_added = True
        return model_numbers
    
    def gear_ratios(self):
        neighbor_models = defaultdict(list)

        for num, neighbor in self._model_numbers():
            neighbor_models[neighbor].append(num)
        gear_ratios = [math.prod(v) for (k, v) in neighbor_models.items() if len(v) > 1]
        # flat_list = [item for sublist in l for item in sublist]
        # return [g for gear in gears for g in gear]
        return gear_ratios
    
    def _has_neighbor_symbol(self, i: int, j: int) -> list[(int, int, int)]:
        neighbors: list((int, int)) = []
        
        STEP = 1

        UPPER_BOUND = i - STEP
        LOWER_BOUND = i + STEP
        LEFT_BOUND = j - STEP
        RIGHT_BOUND = j + STEP

        row_indices = range(UPPER_BOUND, LOWER_BOUND + 1)
        col_indices = range(LEFT_BOUND, RIGHT_BOUND + 1)

        for row_indx in row_indices:
            row_in_schematic = row_indx >= 0 and row_indx < len(self.schematic)
            if not row_in_schematic:
                continue

            cols = []

            # First and last row include all columns
            if row_indx == UPPER_BOUND or row_indx == LOWER_BOUND:
                cols = col_indices
            else:
                cols = [LEFT_BOUND, RIGHT_BOUND]

            for col_indx in cols:
                col_in_schematic = col_indx >= 0 and col_indx < len(self.schematic[row_indx])
                if not col_in_schematic:
                    continue
                value = self.schematic[row_indx][col_indx] 
                if re.match(r'[^\d\.]', value):
                    # print(f'"{value}" is not a digit!')
                    # print(f'"{self.schematic[i][j]}" @ ({i}, {j}) has a special symbol "{value}" @ ({row_indx}, {col_indx})')
                    return (row_indx, col_indx)

                # print(f'found {value if value else '.'} ({row_indx}, {col_indx})')
        # print(f'"{self.schematic[i][j]}" @ ({i}, {j}) didn\'t have a neighbor with a special symbol')
        return False

    def __init__(self, schematic_raw: str) -> None:
        self.schematic = [list(i) for i in schematic_raw.split('\n')]

    def __str__(self) -> str:

        return '\n'.join(''.join(row) for row in self.schematic)
    
e = Engine(example_schematic)
e_a = Engine(puzzle.input_data)
print(f'Solution A: {sum(e_a.model_numbers())}')

e_b = Engine(puzzle.input_data)
print(f'Solution B: {sum(e_b.gear_ratios())}')

Solution A: 498559
Solution B: 72246648
